# Importing the Data

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import datetime



In [12]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data_parse_dt(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], keep_date_col=True))
    return pd.concat(dfs)
        
week_nums = [190601, 190608, 190615]
turnstiles_df = get_data_parse_dt(week_nums)

In [13]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [14]:
turnstiles_df.DATE.value_counts().sort_index()

05/25/2019    28855
05/26/2019    29063
05/27/2019    29068
05/28/2019    28873
05/29/2019    29095
05/30/2019    29190
05/31/2019    29651
06/01/2019    29066
06/02/2019    28968
06/03/2019    30022
06/04/2019    29513
06/05/2019    29484
06/06/2019    29213
06/07/2019    28745
06/08/2019    29030
06/09/2019    29133
06/10/2019    29351
06/11/2019    29113
06/12/2019    30023
06/13/2019    29276
06/14/2019    29008
Name: DATE, dtype: int64

In [15]:
from datetime import datetime as dt

In [ ]:
# mask = ((turnstiles_df["C/A"] == "A002") &
#         (turnstiles_df["UNIT"] == "R051") & 
#         (turnstiles_df["SCP"] == "02-00-00") & 
#         (turnstiles_df["STATION"] == "59 ST"))
# turnstiles_df[mask].head()

In [16]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
145002,N045,R187,01-06-00,81 ST-MUSEUM,2019-06-02 17:00:00,2
144620,N045,R187,01-00-00,81 ST-MUSEUM,2019-06-01 17:00:00,2
144871,N045,R187,01-00-02,81 ST-MUSEUM,2019-06-01 17:00:00,2
434865,R174,R034,00-00-02,125 ST,2019-06-08 00:00:00,2
144746,N045,R187,01-00-01,81 ST-MUSEUM,2019-06-01 17:00:00,2


In [17]:
turnstiles_df.DESC.value_counts()

REGULAR       611516
RECOVR AUD      2224
Name: DESC, dtype: int64

In [18]:
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [19]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-05-25 00:00:00,1
409147,R139,R031,04-00-01,34 ST-PENN STA,2019-06-02 08:00:00,1
409149,R139,R031,04-00-01,34 ST-PENN STA,2019-06-02 16:00:00,1
409150,R139,R031,04-00-01,34 ST-PENN STA,2019-06-02 20:00:00,1
409151,R139,R031,04-00-01,34 ST-PENN STA,2019-06-03 00:00:00,1


In [67]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
entries_df = turnstiles_df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [68]:
entries_daily = (entries_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())

In [69]:
entries_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,05/25/2019,7072456
1,A002,R051,02-00-00,59 ST,05/26/2019,7073114
2,A002,R051,02-00-00,59 ST,05/27/2019,7073837
3,A002,R051,02-00-00,59 ST,05/28/2019,7075260
4,A002,R051,02-00-00,59 ST,05/29/2019,7076789


In [70]:
entries_daily[["PREV_DATE", "PREV_ENTRIES"]] = (entries_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))

C:\Users\Josh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [71]:
# Drop the rows for the earliest date in the df
entries_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [72]:
entries_daily[entries_daily["ENTRIES"] < entries_daily["PREV_ENTRIES"]].head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1618,A025,R023,01-03-01,34 ST-HERALD SQ,05/26/2019,1238366337,05/25/2019,1.238369e+09
1619,A025,R023,01-03-01,34 ST-HERALD SQ,05/27/2019,1238364310,05/26/2019,1.238366e+09
1620,A025,R023,01-03-01,34 ST-HERALD SQ,05/28/2019,1238360334,05/27/2019,1.238364e+09
1621,A025,R023,01-03-01,34 ST-HERALD SQ,05/29/2019,1238356306,05/28/2019,1.238360e+09
1622,A025,R023,01-03-01,34 ST-HERALD SQ,05/30/2019,1238352363,05/29/2019,1.238356e+09


In [73]:
# What's the deal with counter being in reverse
mask = ((entries_df["C/A"] == "A025") & 
(entries_df["UNIT"] == "R023") & 
(entries_df["SCP"] == "01-03-01") & 
(entries_df["STATION"] == "34 ST-HERALD SQ") &
(entries_df["DATE_TIME"].dt.date == datetime.datetime(2019, 5, 26).date()))
entries_df[mask].head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES
3269,2019-05-26 20:00:00,A025,R023,01-03-01,34 ST-HERALD SQ,BDFMNQRW,BMT,05/26/2019,20:00:00,1238366337
3268,2019-05-26 16:00:00,A025,R023,01-03-01,34 ST-HERALD SQ,BDFMNQRW,BMT,05/26/2019,16:00:00,1238366929
3267,2019-05-26 12:00:00,A025,R023,01-03-01,34 ST-HERALD SQ,BDFMNQRW,BMT,05/26/2019,12:00:00,1238367347
3266,2019-05-26 08:00:00,A025,R023,01-03-01,34 ST-HERALD SQ,BDFMNQRW,BMT,05/26/2019,08:00:00,1238367539
3265,2019-05-26 04:00:00,A025,R023,01-03-01,34 ST-HERALD SQ,BDFMNQRW,BMT,05/26/2019,04:00:00,1238367622


In [74]:
# Let's see how many stations have this problem

(entries_daily[entries_daily["ENTRIES"] < entries_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

C/A   UNIT  SCP       STATION        
A025  R023  01-03-01  34 ST-HERALD SQ    20
A043  R462  00-06-00  CANAL ST            1
A049  R088  02-05-00  CORTLANDT ST       18
A066  R118  00-00-00  CANAL ST           20
B020  R263  00-03-00  AVENUE H            1
                                         ..
R523  R147  00-00-04  61 ST WOODSIDE     20
R621  R060  00-03-01  EASTN PKWY-MUSM     2
R622  R123  00-00-00  FRANKLIN AV        20
R646  R110  01-00-01  FLATBUSH AV-B.C    20
R730  R431  00-00-04  EASTCHSTER/DYRE    20
Length: 401, dtype: int64

In [77]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
entries_daily["DAILY_ENTRIES"] = entries_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

1376283 8703.0
371 4317056.0
1790 13967417.0
221 7246254.0
4050694 5902992.0
7246468 4050451.0
5571815 104607574.0
459063 4526857.0
228 1393341.0
1828703861 9687851.0
126 4086499.0
1834 2588198.0
149 1252883.0
580 1650137.0
820 1523029.0
366 1047791.0
840 1251816.0
634 1572569.0
857 3126422.0
1113 1946726.0
1057 1214910.0
291 1309224.0
1484 1063313.0
1317 2409654.0
1060 1093122.0
1061 1461284.0
1472 1054311.0
1088 2725203.0
2095 2041280.0
1 1910177.0
212 1003601.0
531 3209791.0
2452 1395536.0
2094 2286050.0
1958 1696178.0
1977 1058528.0
0 1078800.0
2257 2792918.0
14 1327005.0
393281 1661809141.0
323 7193764.0
330 203977549.0
452993419 1830432.0
774 3000575.0
183 1488278.0


In [79]:
entries_daily.head(10)

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,05/26/2019,7073114,05/25/2019,7072456.0,658.0
2,A002,R051,02-00-00,59 ST,05/27/2019,7073837,05/26/2019,7073114.0,723.0
3,A002,R051,02-00-00,59 ST,05/28/2019,7075260,05/27/2019,7073837.0,1423.0
4,A002,R051,02-00-00,59 ST,05/29/2019,7076789,05/28/2019,7075260.0,1529.0
5,A002,R051,02-00-00,59 ST,05/30/2019,7078273,05/29/2019,7076789.0,1484.0
6,A002,R051,02-00-00,59 ST,05/31/2019,7079862,05/30/2019,7078273.0,1589.0
7,A002,R051,02-00-00,59 ST,06/01/2019,7080817,05/31/2019,7079862.0,955.0
8,A002,R051,02-00-00,59 ST,06/02/2019,7081567,06/01/2019,7080817.0,750.0
9,A002,R051,02-00-00,59 ST,06/03/2019,7082949,06/02/2019,7081567.0,1382.0
10,A002,R051,02-00-00,59 ST,06/04/2019,7084565,06/03/2019,7082949.0,1616.0


,,ENTRIES,PREV_ENTRIES,DAILY_ENTRIES
STATION,DATE,,,
JOURNAL SQUARE,06/07/2019,9046,6107889.0,4449866.0
EXCHANGE PLACE,06/10/2019,11564,11088638.0,4086056.0
PATH NEW WTC,06/04/2019,136428,3841291.0,3744967.0
THIRTY THIRD ST,06/12/2019,21150,15938673.0,3501062.0
GROVE STREET,06/10/2019,18467,4475276.0,2948588.0
NEWARK BM BW,06/05/2019,876669,3751815.0,2884212.0
PAVONIA/NEWPORT,06/10/2019,6872,7327375.0,2854565.0
NEWARK C,06/06/2019,109415,2769246.0,2660477.0
LACKAWANNA,06/11/2019,3684,3400425.0,2393564.0


In [84]:
entries_daily = entries_daily.drop(['PREV_ENTRIES', 'ENTRIES'], axis = 1, errors = 'ignore')

In [87]:
entries_daily = entries_daily.drop(['PREV_DATE'], axis = 1, errors = 'ignore')

In [88]:
entries_daily.head(10)

,C/A,UNIT,SCP,STATION,DATE,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,05/26/2019,658.0
2,A002,R051,02-00-00,59 ST,05/27/2019,723.0
3,A002,R051,02-00-00,59 ST,05/28/2019,1423.0
4,A002,R051,02-00-00,59 ST,05/29/2019,1529.0
5,A002,R051,02-00-00,59 ST,05/30/2019,1484.0
6,A002,R051,02-00-00,59 ST,05/31/2019,1589.0
7,A002,R051,02-00-00,59 ST,06/01/2019,955.0
8,A002,R051,02-00-00,59 ST,06/02/2019,750.0
9,A002,R051,02-00-00,59 ST,06/03/2019,1382.0
10,A002,R051,02-00-00,59 ST,06/04/2019,1616.0


In [93]:
entries_daily.describe()

,DAILY_ENTRIES
count,97067.000000
mean,1583.047503
std,15167.539434
min,0.000000
25%,270.000000
50%,800.000000
75%,1565.000000
max,985931.000000


In [98]:
entries_daily.STATION.nunique()

378

In [ ]:
entries_daily.group_by